In [ ]:
%%shell

git clone https://github.com/ultralytics/yolov5  # clone
cd yolov5
pip install -qr requirements.txt  # install

cp -rf * ../

Cloning into 'yolov5'...
remote: Enumerating objects: 13176, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 13176 (delta 7), reused 8 (delta 4), pack-reused 13156
Receiving objects: 100% (13176/13176), 12.01 MiB | 30.14 MiB/s, done.
Resolving deltas: 100% (9153/9153), done.
     |████████████████████████████████| 596 kB 5.2 MB/s 


# 准备数据

## 转化为yolo支持的数据

In [ ]:
%%shell
if [ -e dataset ]; then
  rm -rf dataset
fi

mkdir dataset
cd dataset
mkdir lung
cd lung 
mkdir images 
mkdir labels
mkdir images/train images/test
mkdir labels/train labels/test

In [ ]:
!pip install -q "labelbox[data]"

     |████████████████████████████████| 165 kB 5.1 MB/s 
     |████████████████████████████████| 10.9 MB 45.7 MB/s 
     |████████████████████████████████| 6.3 MB 56.2 MB/s 


In [ ]:
from labelbox import Client, OntologyBuilder
from labelbox.data.annotation_types import Geometry
from  labelbox.data.annotation_types.collection import LabelList
from PIL import Image
import numpy as np
import os
import torch

In [ ]:
from enum import Enum

class SegClsName:
  VESSEL = "血管"
  BRONCHUS = "支气管"

  def get_all_names():
    return ['支气管', '血管']

In [ ]:
API_KEY = "YOURS"
PROJECT_ID = "YOURS"
client = Client(api_key=API_KEY)
project = client.get_project(PROJECT_ID)
labels = project.label_generator().as_list()

/usr/local/lib/python3.7/dist-packages/labelbox/data/annotation_types/classification/classification.py:85: UserWarning: Dropdown classification is deprecated and will be removed in a future release
  warnings.warn("Dropdown classification is deprecated and will be "


In [ ]:
indices = torch.randperm(len(labels)).tolist()
test_labels = LabelList([labels[i] for i in indices[-5:]])
train_labels = LabelList([labels[i] for i in indices[:-5]])
len(labels), len(train_labels), len(test_labels)

(104, 99, 5)

In [ ]:
import threading

def convert_labelbox_to_yolodataset(lb_labels: LabelList, dir_path):
  def convert(lb):
    img = Image.fromarray(np.uint8(lb.data.value)).convert("RGB")
    img_name = lb.data.external_id
    img.save(os.path.join(dir_path, img_name))
    label_dir = dir_path[::-1].replace("images"[::-1], "labels"[::-1], 1)[::-1]
    label_name = img_name.split(".")[0]+".txt"
    boxes = []
    for a in lb.object_annotations():
      mask = a.value.draw()[:, :, 0]
      height, width = mask.shape
      pos = np.where(mask != 0)
      cls_id = SegClsName.get_all_names().index(a.name)
      xmin = np.min(pos[1])/width
      xmax = np.max(pos[1])/width
      ymin = np.min(pos[0])/height
      ymax = np.max(pos[0])/height
      if xmin < xmax and ymin < ymax:
        # print(boxes)
        boxes.append("%d %f %f %f %f"%(cls_id, (xmin+xmax)/2, (ymin+ymax)/2, (xmax-xmin), (ymax-ymin)))
    if len(boxes) != 0:
      boxes = "\n".join(boxes)
      f = open(os.path.join(label_dir, label_name), 'w')
      f.write(boxes)
      f.close()
  class ConvertThread(threading.Thread):
    def __init__(self, lb):
      threading.Thread.__init__(self)
      self.lb = lb

    def run(self):
      convert(self.lb)

  ths = []
  for lb in lb_labels:
    ths.append(ConvertThread(lb))
  for t in ths: t.start()
  for t in ths: t.join()
# convert_labelbox_to_yolodataset(test_labels, "/content/dataset/lung/images")

In [ ]:
def write_yolodataset_config(cfg_file, data_path, train_path, test_path, n_cls, cls_names):
  dataset_config = '''# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: %(path)s  # dataset root dir
train: %(train_path)s  # train images (relative to 'path') 128 images
val: %(train_path)s  # val images (relative to 'path') 128 images
test: %(test_path)s # test images (optional)

# Classes
nc: %(nc)d  # number of classes
names: %(cls_names)s  # class names'''%{"path": data_path, "train_path": train_path, "test_path": test_path, "nc": n_cls, "cls_names": str(cls_names)}

  f = open(cfg_file, 'w')
  f.write(dataset_config)
  f.close()

In [ ]:
convert_labelbox_to_yolodataset(train_labels, "./dataset/lung/images/train")
convert_labelbox_to_yolodataset(test_labels, "./dataset/lung/images/test")
write_yolodataset_config("lung.yml", "/content/dataset/lung", "images/train", "images/test", len(SegClsName.get_all_names()), ["bronchus","vessel"])

In [ ]:
import train as train

train.run(img=640, batch=16, epochs=50, data='/content/lung.yml', weights='yolov5s.pt')

train: weights=yolov5s.pt, cfg=, data=/content/lung.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, img=640, batch=16
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5



requirements: 1 package updated per /content/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-5-4 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
Weights & Biases: run 'pip install wandb' to automatically track and visualize YOLOv5 🚀 runs (RECOMMENDED)
TensorBoard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/


  0%|          | 0.00/755k [00:00<?, ?B/s]

  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Overriding model.yaml nc=80 with nc=2

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]            

Namespace(artifact_alias='latest', batch=16, batch_size=16, bbox_interval=-1, bucket='', cache=None, cfg='', cos_lr=False, data='/content/lung.yml', device='', entity=None, epochs=50, evolve=None, exist_ok=False, freeze=[0], hyp='data/hyps/hyp.scratch-low.yaml', image_weights=False, img=640, imgsz=640, label_smoothing=0.0, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, noplots=False, nosave=False, noval=False, optimizer='SGD', patience=100, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', save_period=-1, single_cls=False, sync_bn=False, upload_dataset=False, weights='yolov5s.pt', workers=8)

In [ ]:
import torch

model = torch.hub.load('/content', 'custom', path='/content/best.pt', source='local')



requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /content/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-5-4 torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)



Exception: ignored

In [ ]:
from PIL import Image

im = Image.open('/content/dataset/lung/images/train/1-100-2.jpg')
results = model(im)

In [ ]:
results.pandas().xyxy[0]

In [ ]:
for idx,data in results.pandas().xyxy[0].iterrows():
  print(idx, type(data['xmin']), type(0.01), data['ymin'], data['xmax'], data['ymax'], data['class'])

In [ ]:
im

In [ ]:
im.crop((534.607544,	676.614258,	693.596680,	858.074585))

In [ ]:
Image.fromarray(np.array(im)[:600, :300])